“Data Page: Gross domestic product (GDP)”. Our World in Data (2024). Data adapted from World Bank and OECD. Retrieved from https://ourworldindata.org/grapher/national-gdp-constant-usd-wb [online resource]

In [1]:
import pandas as pd
import pickle
import seaborn as sns
import json
import numpy as np
import pycountry

from matplotlib import pyplot as plt
from math import nan
from scipy.stats import spearmanr

In [2]:
with open('data/full_user_data.pkl', 'rb') as file:
    user_data = pickle.load(file)

user_data

,steamid,communityvisibilitystate,personaname,profileurl,avatar,avatarmedium,avatarfull,avatarhash,personastate,primaryclanid,...,realname,loccountrycode,locstatecode,loccityid,gameserverip,gameserversteamid,gameextrainfo,gameid,lobbysteamid,lastlogoff
76561199016387141,76561199016387141,3,leandro,https://steamcommunity.com/profiles/7656119901...,https://avatars.steamstatic.com/fef49e7fa7e199...,https://avatars.steamstatic.com/fef49e7fa7e199...,https://avatars.steamstatic.com/fef49e7fa7e199...,fef49e7fa7e1997310d705b2a6158ff8dc1cdfeb,0,103582791429521408,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
76561198211968339,76561198211968339,3,r.relu33,https://steamcommunity.com/profiles/7656119821...,https://avatars.steamstatic.com/001e645f7caf7e...,https://avatars.steamstatic.com/001e645f7caf7e...,https://avatars.steamstatic.com/001e645f7caf7e...,001e645f7caf7e3fe65835f1c180505e930cb40d,0,103582791429521408,...,paulica,RO,34,38638.0,NaN,NaN,NaN,NaN,NaN,NaN
76561198281739227,76561198281739227,3,StownDiaz,https://steamcommunity.com/id/stownmx/,https://avatars.steamstatic.com/312795cc543919...,https://avatars.steamstatic.com/312795cc543919...,https://avatars.steamstatic.com/312795cc543919...,312795cc543919ce0354af5ba04ec23334dc3c0c,0,103582791441353032,...,Stown,MX,09,NaN,NaN,NaN,NaN,NaN,NaN,NaN
76561199348301309,76561199348301309,3,珂波,https://steamcommunity.com/profiles/7656119934...,https://avatars.steamstatic.com/f56a3221a16882...,https://avatars.steamstatic.com/f56a3221a16882...,https://avatars.steamstatic.com/f56a3221a16882...,f56a3221a168821d8439a460e5993f0c758115bd,0,103582791429521408,...,张佳宁,CN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
76561199662454569,76561199662454569,3,jjukbkh94,https://steamcommunity.com/profiles/7656119966...,https://avatars.steamstatic.com/fef49e7fa7e199...,https://avatars.steamstatic.com/fef49e7fa7e199...,https://avatars.steamstatic.com/fef49e7fa7e199...,fef49e7fa7e1997310d705b2a6158ff8dc1cdfeb,0,103582791429521408,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76561198093803237,76561198093803237,3,Lemons4me,https://steamcommunity.com/profiles/7656119809...,https://avatars.steamstatic.com/1286efc260c1e9...,https://avatars.steamstatic.com/1286efc260c1e9...,https://avatars.steamstatic.com/1286efc260c1e9...,1286efc260c1e9fbeec1cd8e03638f8c388b3b0f,1,103582791436588017,...,Dog = God,NaN,NaN,NaN,NaN,90209463684485130,Deadlock,1422450,NaN,NaN
76561198087747419,76561198087747419,3,Dipper,https://steamcommunity.com/id/Pinetree_Dipper/,https://avatars.steamstatic.com/70da20b3bf8ba4...,https://avatars.steamstatic.com/70da20b3bf8ba4...,https://avatars.steamstatic.com/70da20b3bf8ba4...,70da20b3bf8ba4a09734ed393b73d6a2698da573,0,103582791456814001,...,Dipper,DE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
76561198836088840,76561198836088840,3,Flowy,https://steamcommunity.com/id/kereme/,https://avatars.steamstatic.com/2e728e5536134d...,https://avatars.steamstatic.com/2e728e5536134d...,https://avatars.steamstatic.com/2e728e5536134d...,2e728e5536134dd9f8775f7add79e03eca4860a0,0,103582791429521408,...,NaN,TR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
76561198101652863,76561198101652863,2,Ciker.,https://steamcommunity.com/id/junho3004/,https://avatars.steamstatic.com/33281ac55fcf4d...,https://avatars.steamstatic.com/33281ac55fcf4d...,https://avatars.steamstatic.com/33281ac55fcf4d...,33281ac55fcf4d01c0d75fb312b8325361adad64,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
gdps = pd.read_csv('data/gdp/national-gdp-constant-usd-wb.csv')
gdps

,Entity,Code,Year,GDP (constant 2015 US$),time
0,Afghanistan,NaN,NaN,1.634520e+10,2021
1,Albania,ALB,2022.0,1.431979e+10,2022
2,Algeria,DZA,2022.0,1.796020e+11,2022
3,American Samoa,ASM,2022.0,6.627252e+08,2022
4,Andorra,AND,2022.0,3.148860e+09,2022
...,...,...,...,...,...
219,Vietnam,VNM,2022.0,3.589184e+11,2022
220,World,OWID_WRL,2022.0,8.999466e+13,2022
221,Yemen,NaN,NaN,3.678914e+10,2018
222,Zambia,ZMB,2022.0,2.618516e+10,2022


In [4]:
curated = user_data.copy()
# Step 1: Clean and preprocess the 'curated' DataFrame

# Drop rows where 'loccountrycode' is None or NaN
curated_clean = curated.dropna(subset=['loccountrycode'])

# Map 'loccountrycode' to country names using pycountry
def code_to_country_name(code):
    try:
        return pycountry.countries.get(alpha_2=code.upper()).name
    except:
        return np.nan

curated_clean['country'] = curated_clean['loccountrycode'].apply(code_to_country_name)

# Drop rows where the country name couldn't be found
curated_clean = curated_clean.dropna(subset=['country'])

# Step 2: Calculate the proportion of VAC-banned users per country

# Ensure 'VACBanned' is a boolean column
curated_clean['VACBanned'] = curated_clean['VACBanned'].astype(bool)

# Group by 'country' and compute VAC ban statistics
vac_grouped = curated_clean.groupby('country')['VACBanned']

# Count of VAC-banned users per country
vac_banned_counts = vac_grouped.sum()

# Total number of users per country
total_users = vac_grouped.count()

# Proportion of VAC-banned users per country
proportion_vac_banned = (vac_banned_counts / total_users).reset_index()
proportion_vac_banned.columns = ['country', 'proportion_vac_banned']

# Step 3: Prepare the 'gdps' DataFrame

# Optionally, ensure country names in 'gdps' are standardized
def standardized_country_name(name):
    try:
        country = pycountry.countries.search_fuzzy(name)[0]
        return country.name
    except:
        return np.nan

gdps['standardized_country'] = gdps['Entity'].apply(standardized_country_name)

# Drop rows where the country name couldn't be standardized
gdps_clean = gdps.dropna(subset=['standardized_country'])

# Select relevant columns and rename for clarity
gdps_clean = gdps_clean[['standardized_country', 'GDP (constant 2015 US$)']]
gdps_clean.columns = ['country', 'GDP']

# Step 4: Merge the DataFrames on 'country'

# Merge the two DataFrames on 'country'
merged_df = pd.merge(proportion_vac_banned, gdps_clean, on='country', how='inner')

# Step 5: Remove any remaining NaN values
merged_df = merged_df.dropna(subset=['proportion_vac_banned', 'GDP'])

# Step 6: Compute the Spearman rank correlation
corr_coefficient, p_value = spearmanr(merged_df['proportion_vac_banned'], merged_df['GDP'])

# Print the results
print("Spearman's rank correlation coefficient:", corr_coefficient)
print("p-value:", p_value)

C:\Users\dennm\AppData\Local\Temp\ipykernel_78764\793694090.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  curated_clean['country'] = curated_clean['loccountrycode'].apply(code_to_country_name)


Spearman's rank correlation coefficient: -0.05989600089176376
p-value: 0.4007014775940232


Assuming a confidence threshold of 0.05, the p-value 0.401 > 0.05 is too high, thus we cannot reject the null hypothesis that the GDP and proportion of cheaters are unrelated.